In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [2]:
# Load the three datasets
demographics_df = pd.read_csv('/Users/chadleyayco/Documents/MML/customer_demographics_contaminated.csv')
transactions_df = pd.read_csv('/Users/chadleyayco/Documents/MML/customer_transactions_contaminated.csv')
interactions_df = pd.read_csv('/Users/chadleyayco/Documents/MML/social_media_interactions_contaminated.csv')

print("Dataset shapes:")
print(f"Demographics: {demographics_df.shape}")
print(f"Transactions: {transactions_df.shape}")
print(f"Interactions: {interactions_df.shape}")

Dataset shapes:
Demographics: (3200, 6)
Transactions: (3200, 6)
Interactions: (3200, 6)


In [3]:
# Display basic information about each dataset
print("=== DEMOGRAPHICS DATASET ===")
print(demographics_df.info())
print("\nFirst few rows:")
print(demographics_df.head())

print("\n=== TRANSACTIONS DATASET ===")
print(transactions_df.info())
print("\nFirst few rows:")
print(transactions_df.head())

print("\n=== INTERACTIONS DATASET ===")
print(interactions_df.info())
print("\nFirst few rows:")
print(interactions_df.head())

=== DEMOGRAPHICS DATASET ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3200 entries, 0 to 3199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerID   3200 non-null   object
 1   Age          2909 non-null   object
 2   Gender       3200 non-null   object
 3   Location     3200 non-null   object
 4   IncomeLevel  2897 non-null   object
 5   SignupDate   3200 non-null   object
dtypes: object(6)
memory usage: 150.1+ KB
None

First few rows:
                             CustomerID   Age  Gender           Location  \
0  9207fa75-5758-48d1-94ad-19c041e0520f  51.0  Female         Jensenberg   
1  5fb09cd8-a473-46f7-80bd-6e49cf509078   NaN  Female       Castilloport   
2  c139496e-cc89-498a-bd90-1fb4627b6cff  37.0    Male  Lake Jennifertown   
3  50118139-7264-428f-81cc-a25fddc5d6dd  44.0    Male          Port Carl   
4  7d1f2bbc-8d16-4fbc-9b37-ece3324e8ed4  50.0  Female          Jessebury   

  IncomeLevel  S

In [4]:
def analyze_missing_values(df, dataset_name):
    """Analyze missing values in a dataset"""
    print(f"\n=== {dataset_name} - Missing Values Analysis ===")
    missing_data = df.isnull().sum()
    missing_percent = (missing_data / len(df)) * 100
    
    missing_table = pd.DataFrame({
        'Missing Count': missing_data,
        'Percentage': missing_percent
    })
    
    missing_table = missing_table[missing_table['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
    
    if not missing_table.empty:
        print(missing_table)
    else:
        print("No missing values found!")
    
    return missing_table

# Analyze missing values for each dataset
demo_missing = analyze_missing_values(demographics_df, "DEMOGRAPHICS")
trans_missing = analyze_missing_values(transactions_df, "TRANSACTIONS")
inter_missing = analyze_missing_values(interactions_df, "INTERACTIONS")


=== DEMOGRAPHICS - Missing Values Analysis ===
             Missing Count  Percentage
IncomeLevel            303     9.46875
Age                    291     9.09375

=== TRANSACTIONS - Missing Values Analysis ===
                 Missing Count  Percentage
Amount                     304     9.50000
ProductCategory            299     9.34375

=== INTERACTIONS - Missing Values Analysis ===
           Missing Count  Percentage
Sentiment            329    10.28125
Platform             311     9.71875


In [6]:
# Create copies for cleaning
demographics_clean = demographics_df.copy()
transactions_clean = transactions_df.copy()
interactions_clean = interactions_df.copy()

print("=== HANDLING MISSING VALUES ===")

# First, convert data types properly before handling missing values
print("Converting data types...")

# Convert Age to numeric (handles string numbers and sets invalid to NaN)
demographics_clean['Age'] = pd.to_numeric(demographics_clean['Age'], errors='coerce')

# Convert Amount to numeric (handles string numbers and sets invalid to NaN)
transactions_clean['Amount'] = pd.to_numeric(transactions_clean['Amount'], errors='coerce')

# Demographics - Handle Age missing values with median imputation
median_age = demographics_clean['Age'].median()
demographics_clean['Age'].fillna(median_age, inplace=True)
print(f"Age missing values filled with median: {median_age:.1f}")

# Demographics - Handle IncomeLevel missing values with mode
# Replace empty strings with NaN first
demographics_clean['IncomeLevel'] = demographics_clean['IncomeLevel'].replace('', np.nan)
mode_income = demographics_clean['IncomeLevel'].mode()[0]
demographics_clean['IncomeLevel'].fillna(mode_income, inplace=True)
print(f"IncomeLevel missing values filled with mode: {mode_income}")

# Transactions - Handle Amount missing values with median
median_amount = transactions_clean['Amount'].median()
transactions_clean['Amount'].fillna(median_amount, inplace=True)
print(f"Amount missing values filled with median: {median_amount:.2f}")

# Transactions - Handle ProductCategory missing values with mode
# Replace empty strings with NaN first
transactions_clean['ProductCategory'] = transactions_clean['ProductCategory'].replace('', np.nan)
mode_category = transactions_clean['ProductCategory'].mode()[0]
transactions_clean['ProductCategory'].fillna(mode_category, inplace=True)
print(f"ProductCategory missing values filled with mode: {mode_category}")

# Interactions - Handle Platform missing values with mode
# Replace empty strings with NaN first
interactions_clean['Platform'] = interactions_clean['Platform'].replace('', np.nan)
mode_platform = interactions_clean['Platform'].mode()[0]
interactions_clean['Platform'].fillna(mode_platform, inplace=True)
print(f"Platform missing values filled with mode: {mode_platform}")

# Interactions - Handle Sentiment missing values with mode
# Replace empty strings with NaN first
interactions_clean['Sentiment'] = interactions_clean['Sentiment'].replace('', np.nan)
mode_sentiment = interactions_clean['Sentiment'].mode()[0]
interactions_clean['Sentiment'].fillna(mode_sentiment, inplace=True)
print(f"Sentiment missing values filled with mode: {mode_sentiment}")

=== HANDLING MISSING VALUES ===
Converting data types...
Age missing values filled with median: 45.0
IncomeLevel missing values filled with mode: High
Amount missing values filled with median: 497.28
ProductCategory missing values filled with mode: Clothing
Platform missing values filled with mode: Instagram
Sentiment missing values filled with mode: Positive


In [7]:
print("\n=== DUPLICATE ANALYSIS ===")

# Check for duplicates
print(f"Demographics duplicates: {demographics_clean.duplicated().sum()}")
print(f"Transactions duplicates: {transactions_clean.duplicated().sum()}")
print(f"Interactions duplicates: {interactions_clean.duplicated().sum()}")

# Remove duplicates if any exist
demographics_clean = demographics_clean.drop_duplicates()
transactions_clean = transactions_clean.drop_duplicates()
interactions_clean = interactions_clean.drop_duplicates()

print("Duplicates removed (if any existed)")


=== DUPLICATE ANALYSIS ===
Demographics duplicates: 177
Transactions duplicates: 185
Interactions duplicates: 181
Duplicates removed (if any existed)


In [9]:
print("\n=== DATA TYPE STANDARDIZATION ===")

# Convert date columns to datetime with flexible parsing
print("Converting date columns...")

# Handle mixed date formats - some might be YYYY-MM-DD, others DD/MM/YYYY
demographics_clean['SignupDate'] = pd.to_datetime(demographics_clean['SignupDate'], 
                                                 format='mixed', 
                                                 dayfirst=True, 
                                                 errors='coerce')

transactions_clean['TransactionDate'] = pd.to_datetime(transactions_clean['TransactionDate'], 
                                                      format='mixed', 
                                                      dayfirst=True, 
                                                      errors='coerce')

interactions_clean['InteractionDate'] = pd.to_datetime(interactions_clean['InteractionDate'], 
                                                      format='mixed', 
                                                      dayfirst=True, 
                                                      errors='coerce')

print("Date conversion completed")

# Check for any failed date conversions
date_errors = {
    'SignupDate': demographics_clean['SignupDate'].isna().sum(),
    'TransactionDate': transactions_clean['TransactionDate'].isna().sum(),
    'InteractionDate': interactions_clean['InteractionDate'].isna().sum()
}

print("Date conversion errors (NaT values):", date_errors)

# Ensure numeric columns are properly typed (already done in 4.1, but double-checking)
demographics_clean['Age'] = pd.to_numeric(demographics_clean['Age'], errors='coerce')
transactions_clean['Amount'] = pd.to_numeric(transactions_clean['Amount'], errors='coerce')

# Standardize categorical columns (strip whitespace, proper case)
print("Standardizing categorical columns...")

categorical_cols_demo = ['Gender', 'Location', 'IncomeLevel']
for col in categorical_cols_demo:
    # Handle NaN values and convert to string safely
    demographics_clean[col] = demographics_clean[col].fillna('Unknown').astype(str).str.strip().str.title()

categorical_cols_trans = ['ProductCategory', 'PaymentMethod']
for col in categorical_cols_trans:
    # Handle NaN values and convert to string safely
    transactions_clean[col] = transactions_clean[col].fillna('Unknown').astype(str).str.strip().str.title()

categorical_cols_inter = ['Platform', 'InteractionType', 'Sentiment']
for col in categorical_cols_inter:
    # Handle NaN values and convert to string safely
    interactions_clean[col] = interactions_clean[col].fillna('Unknown').astype(str).str.strip().str.title()

print("Data types and formats standardized")

# Display updated data types
print("\nUpdated data types:")
print("Demographics:", demographics_clean.dtypes)
print("Transactions:", transactions_clean.dtypes)
print("Interactions:", interactions_clean.dtypes)

# Show date ranges to verify conversion worked
print("\nDate ranges after conversion:")
print(f"SignupDate: {demographics_clean['SignupDate'].min()} to {demographics_clean['SignupDate'].max()}")
print(f"TransactionDate: {transactions_clean['TransactionDate'].min()} to {transactions_clean['TransactionDate'].max()}")
print(f"InteractionDate: {interactions_clean['InteractionDate'].min()} to {interactions_clean['InteractionDate'].max()}")


=== DATA TYPE STANDARDIZATION ===
Converting date columns...
Date conversion completed
Date conversion errors (NaT values): {'SignupDate': np.int64(0), 'TransactionDate': np.int64(0), 'InteractionDate': np.int64(0)}
Standardizing categorical columns...
Data types and formats standardized

Updated data types:
Demographics: CustomerID             object
Age                   float64
Gender                 object
Location               object
IncomeLevel            object
SignupDate     datetime64[ns]
dtype: object
Transactions: CustomerID                 object
TransactionID              object
TransactionDate    datetime64[ns]
Amount                    float64
ProductCategory            object
PaymentMethod              object
dtype: object
Interactions: CustomerID                 object
InteractionID              object
InteractionDate    datetime64[ns]
Platform                   object
InteractionType            object
Sentiment                  object
dtype: object

Date ranges afte

In [10]:
# Sort transactions by TransactionDate in ascending order
transactions_clean = transactions_clean.sort_values('TransactionDate').reset_index(drop=True)
print("Transactions sorted by TransactionDate in ascending order")
print("Date range:", transactions_clean['TransactionDate'].min(), "to", transactions_clean['TransactionDate'].max())

Transactions sorted by TransactionDate in ascending order
Date range: 2022-07-01 00:00:00 to 2024-06-30 00:00:00


In [11]:
print("\n=== DATA INTEGRITY VALIDATION ===")

# Check for negative amounts
negative_amounts = transactions_clean[transactions_clean['Amount'] < 0]
print(f"Transactions with negative amounts: {len(negative_amounts)}")

# Check for unrealistic ages
unrealistic_ages = demographics_clean[(demographics_clean['Age'] < 0) | (demographics_clean['Age'] > 120)]
print(f"Records with unrealistic ages: {len(unrealistic_ages)}")

# Check for future dates (beyond today)
today = datetime.now()
future_signups = demographics_clean[demographics_clean['SignupDate'] > today]
future_transactions = transactions_clean[transactions_clean['TransactionDate'] > today]
future_interactions = interactions_clean[interactions_clean['InteractionDate'] > today]

print(f"Future signup dates: {len(future_signups)}")
print(f"Future transaction dates: {len(future_transactions)}")
print(f"Future interaction dates: {len(future_interactions)}")

# Validate CustomerID consistency across datasets
unique_customers_demo = set(demographics_clean['CustomerID'])
unique_customers_trans = set(transactions_clean['CustomerID'])
unique_customers_inter = set(interactions_clean['CustomerID'])

print(f"\nUnique customers in demographics: {len(unique_customers_demo)}")
print(f"Unique customers in transactions: {len(unique_customers_trans)}")
print(f"Unique customers in interactions: {len(unique_customers_inter)}")

# Find customers who exist in all datasets
common_customers = unique_customers_demo.intersection(unique_customers_trans).intersection(unique_customers_inter)
print(f"Customers present in all three datasets: {len(common_customers)}")


=== DATA INTEGRITY VALIDATION ===
Transactions with negative amounts: 40
Records with unrealistic ages: 63
Future signup dates: 0
Future transaction dates: 0
Future interaction dates: 0

Unique customers in demographics: 3000
Unique customers in transactions: 1871
Unique customers in interactions: 1893
Customers present in all three datasets: 1181


In [12]:
print("\n=== MERGING DATASETS ===")

# Start with demographics as the base
merged_df = demographics_clean.copy()

# Merge with transactions (left join to keep all customers)
merged_df = merged_df.merge(transactions_clean, on='CustomerID', how='left', suffixes=('', '_trans'))

# Merge with interactions (left join to keep all customers)
merged_df = merged_df.merge(interactions_clean, on='CustomerID', how='left', suffixes=('', '_inter'))

print(f"Final merged dataset shape: {merged_df.shape}")
print("Columns in merged dataset:", list(merged_df.columns))

# Display first few rows of merged data
print("\nFirst 5 rows of merged dataset:")
print(merged_df.head())


=== MERGING DATASETS ===
Final merged dataset shape: (5728, 16)
Columns in merged dataset: ['CustomerID', 'Age', 'Gender', 'Location', 'IncomeLevel', 'SignupDate', 'TransactionID', 'TransactionDate', 'Amount', 'ProductCategory', 'PaymentMethod', 'InteractionID', 'InteractionDate', 'Platform', 'InteractionType', 'Sentiment']

First 5 rows of merged dataset:
                             CustomerID   Age  Gender      Location  \
0  9207fa75-5758-48d1-94ad-19c041e0520f  51.0  Female    Jensenberg   
1  9207fa75-5758-48d1-94ad-19c041e0520f  51.0  Female    Jensenberg   
2  9207fa75-5758-48d1-94ad-19c041e0520f  51.0  Female    Jensenberg   
3  9207fa75-5758-48d1-94ad-19c041e0520f  51.0  Female    Jensenberg   
4  5fb09cd8-a473-46f7-80bd-6e49cf509078  45.0  Female  Castilloport   

  IncomeLevel SignupDate                         TransactionID  \
0         Low 2022-11-17  d4922534-dfd9-47d9-8fc8-d0421284b682   
1         Low 2022-11-17  d4922534-dfd9-47d9-8fc8-d0421284b682   
2         Low 2

In [13]:
print("\n=== NUMERIC VARIABLES STATISTICS ===")

numeric_columns = merged_df.select_dtypes(include=[np.number]).columns
print("Numeric columns:", list(numeric_columns))

# Calculate basic statistics
stats_summary = merged_df[numeric_columns].describe()
print("\nBasic Statistics:")
print(stats_summary)


=== NUMERIC VARIABLES STATISTICS ===
Numeric columns: ['Age', 'Amount']

Basic Statistics:
               Age       Amount
count  5728.000000  4159.000000
mean     45.388617   492.543123
std      18.548060   284.824830
min      -1.000000  -100.000000
25%      33.000000   256.465000
50%      45.000000   497.285000
75%      57.000000   725.710000
max     150.000000   999.860000


In [14]:
print("\n=== CATEGORICAL VARIABLES DISTRIBUTIONS ===")

categorical_columns = merged_df.select_dtypes(include=['object', 'category']).columns
categorical_columns = [col for col in categorical_columns if col not in ['CustomerID', 'TransactionID', 'InteractionID']]

for col in categorical_columns:
    if col in merged_df.columns:
        print(f"\n{col} distribution:")
        print(merged_df[col].value_counts().head(10))
        print(f"Unique values: {merged_df[col].nunique()}")


=== CATEGORICAL VARIABLES DISTRIBUTIONS ===

Gender distribution:
Gender
Female    2908
Male      2820
Name: count, dtype: int64
Unique values: 2

Location distribution:
Location
West David             20
New Anthonyland        18
Port Austin            16
Johnsonburgh           15
East Annaview          15
Emilyville             15
South Samanthaburgh    15
Robinburgh             13
Johnville              13
Joannaville            12
Name: count, dtype: int64
Unique values: 2696

IncomeLevel distribution:
IncomeLevel
High      2333
Low       1726
Medium    1669
Name: count, dtype: int64
Unique values: 3

ProductCategory distribution:
ProductCategory
Clothing           1156
Electronics         792
Automotive          772
Health & Beauty     732
Home & Garden       707
Name: count, dtype: int64
Unique values: 5

PaymentMethod distribution:
PaymentMethod
Debit Card       1073
Paypal           1047
Credit Card      1038
Bank Transfer    1001
Name: count, dtype: int64
Unique values: 4

Pl

In [15]:
print("\n=== OUTLIER ANALYSIS ===")

# Function to detect outliers using IQR method
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Analyze outliers for numeric columns
for col in numeric_columns:
    if col in merged_df.columns and not merged_df[col].isna().all():
        outliers, lower, upper = detect_outliers_iqr(merged_df, col)
        print(f"\n{col}:")
        print(f"  Outliers detected: {len(outliers)}")
        print(f"  Normal range: {lower:.2f} to {upper:.2f}")
        if len(outliers) > 0:
            print(f"  Outlier range: {outliers[col].min():.2f} to {outliers[col].max():.2f}")


=== OUTLIER ANALYSIS ===

Age:
  Outliers detected: 58
  Normal range: -3.00 to 93.00
  Outlier range: 150.00 to 150.00

Amount:
  Outliers detected: 0
  Normal range: -447.40 to 1429.58


In [20]:
# Fix negative amounts (likely refunds - convert to positive or flag separately)
negative_transactions = merged_df[merged_df['Amount'] < 0]
print("Negative transactions to investigate:")
print(negative_transactions[['CustomerID', 'Amount', 'ProductCategory', 'TransactionDate']])

# Fix unrealistic ages
unrealistic_ages = merged_df[(merged_df['Age'] < 0) | (merged_df['Age'] > 100)]
print("Unrealistic ages to investigate:")
print(unrealistic_ages[['CustomerID', 'Age', 'Gender', 'Location']])

Negative transactions to investigate:
                                CustomerID  Amount  ProductCategory  \
60    c56a1423-e1b0-45c1-974a-55f5fa029c88  -100.0      Electronics   
83    a800ccba-e1ca-4f52-ad9f-14f3432699c9  -100.0         Clothing   
84    a800ccba-e1ca-4f52-ad9f-14f3432699c9  -100.0         Clothing   
229   b7b84b81-c61b-4f23-b3db-b18010484122  -100.0         Clothing   
230   b7b84b81-c61b-4f23-b3db-b18010484122  -100.0         Clothing   
...                                    ...     ...              ...   
5377  a94e3f55-2f0e-432e-8706-9815c4371376  -100.0    Home & Garden   
5631  f35c6e3f-2fac-4b7a-b713-863a0a37160c  -100.0  Health & Beauty   
5720  7eb7f7df-9b0a-416a-829e-bcf4d8bd57dc  -100.0       Automotive   
5721  7eb7f7df-9b0a-416a-829e-bcf4d8bd57dc  -100.0       Automotive   
5722  7eb7f7df-9b0a-416a-829e-bcf4d8bd57dc  -100.0       Automotive   

     TransactionDate  
60        2024-05-31  
83        2023-09-25  
84        2023-09-25  
229       2023-03

In [3]:
import pandas as pd
import numpy as np

# Load datasets
merged_df = pd.read_csv('customer_data_cleaned_merged.csv')
demographics_clean = pd.read_csv('customer_demographics_cleaned.csv')

print("=== AGE OUTLIER CORRECTION ===")
print("Correcting age outliers based on group criteria:")
print("- Ages < 15 years (too young for typical customers)")
print("- Ages > 95 years (unrealistic for active customers)")

# Identify outliers before correction
age_outliers_before = merged_df[(merged_df['Age'] < 15) | (merged_df['Age'] > 95)]
print(f"\nRecords with unrealistic ages before correction: {len(age_outliers_before)}")

# Calculate median age from realistic values only
realistic_ages = merged_df[(merged_df['Age'] >= 15) & (merged_df['Age'] <= 95)]['Age']
median_realistic_age = realistic_ages.median()
print(f"\nMedian age (from realistic values only): {median_realistic_age:.1f}")

# Apply correction
merged_df.loc[(merged_df['Age'] < 15) | (merged_df['Age'] > 95), 'Age'] = median_realistic_age
demographics_clean.loc[(demographics_clean['Age'] < 15) | (demographics_clean['Age'] > 95), 'Age'] = median_realistic_age

# Verify correction
age_outliers_after = merged_df[(merged_df['Age'] < 15) | (merged_df['Age'] > 95)]
print(f"Records with unrealistic ages after correction: {len(age_outliers_after)}")
print("✓ Age outlier correction completed")

=== AGE OUTLIER CORRECTION ===
Correcting age outliers based on group criteria:
- Ages < 15 years (too young for typical customers)
- Ages > 95 years (unrealistic for active customers)

Records with unrealistic ages before correction: 117

Median age (from realistic values only): 45.0
Records with unrealistic ages after correction: 0
✓ Age outlier correction completed


In [4]:
print("\n=== FINAL OUTLIER TREATMENT SUMMARY ===")

# Negative and zero amounts analysis
negative_zero_amounts = merged_df[merged_df['Amount'] <= 0]
print(f"Transactions with negative/zero amounts: {len(negative_zero_amounts)}")
if len(negative_zero_amounts) > 0:
    print("Amount distribution for negative/zero values:")
    print(negative_zero_amounts['Amount'].describe())
    print("\n✓ Decision: Negative/zero amounts retained (treated as refunds/adjustments)")

# Blank fields analysis
total_records = len(merged_df)
records_with_transactions = len(merged_df[merged_df['TransactionID'].notna()])
records_with_interactions = len(merged_df[merged_df['InteractionID'].notna()])
records_with_both = len(merged_df[(merged_df['TransactionID'].notna()) & (merged_df['InteractionID'].notna())])

print(f"\nData completeness after cleaning:")
print(f"Total customer records: {total_records:,}")
print(f"Records with transaction data: {records_with_transactions:,} ({records_with_transactions/total_records*100:.1f}%)")
print(f"Records with interaction data: {records_with_interactions:,} ({records_with_interactions/total_records*100:.1f}%)")
print(f"Records with complete data: {records_with_both:,} ({records_with_both/total_records*100:.1f}%)")
print("✓ Decision: Blank fields retained (valid business scenario)")

print(f"\n=== OUTLIER TREATMENT DECISIONS ===")
print("✅ TREATED AS OUTLIERS:")
print("   • Unrealistic ages (<15, >95) → Replaced with median (45.0)")
print("\n❌ NOT TREATED AS OUTLIERS:")
print("   • Negative/zero amounts → Retained (business refunds/adjustments)")
print("   • Blank transaction/interaction fields → Retained (valid customer states)")


=== FINAL OUTLIER TREATMENT SUMMARY ===
Transactions with negative/zero amounts: 105
Amount distribution for negative/zero values:
count    105.000000
mean     -56.190476
std       49.853265
min     -100.000000
25%     -100.000000
50%     -100.000000
75%        0.000000
max        0.000000
Name: Amount, dtype: float64

✓ Decision: Negative/zero amounts retained (treated as refunds/adjustments)

Data completeness after cleaning:
Total customer records: 5,728
Records with transaction data: 4,159 (72.6%)
Records with interaction data: 4,179 (73.0%)
Records with complete data: 3,030 (52.9%)
✓ Decision: Blank fields retained (valid business scenario)

=== OUTLIER TREATMENT DECISIONS ===
✅ TREATED AS OUTLIERS:
   • Unrealistic ages (<15, >95) → Replaced with median (45.0)

❌ NOT TREATED AS OUTLIERS:
   • Negative/zero amounts → Retained (business refunds/adjustments)
   • Blank transaction/interaction fields → Retained (valid customer states)


In [7]:
demographics_clean = pd.read_csv('customer_demographics_cleaned.csv')

# Apply age correction
realistic_ages = demographics_clean[(demographics_clean['Age'] >= 15) & (demographics_clean['Age'] <= 95)]['Age']
median_realistic_age = realistic_ages.median()
demographics_clean.loc[(demographics_clean['Age'] < 15) | (demographics_clean['Age'] > 95), 'Age'] = median_realistic_age

# Save updated demographics
demographics_clean.to_csv('customer_demographics_cleaned_final.csv', index=False)

import shutil
shutil.copy('customer_transactions_cleaned.csv', 'customer_transactions_cleaned_final.csv')
shutil.copy('social_media_interactions_cleaned.csv', 'social_media_interactions_cleaned_final.csv')

print("All individual datasets updated and saved!")

All individual datasets updated and saved!


In [17]:
# Save the cleaned and merged dataset
output_filename = 'customer_data_cleaned_merged.csv'
merged_df.to_csv(output_filename, index=False)
print(f"\nCleaned and merged dataset saved as: {output_filename}")

# Save individual cleaned datasets as well
demographics_clean.to_csv('customer_demographics_cleaned.csv', index=False)
transactions_clean.to_csv('customer_transactions_cleaned.csv', index=False)
interactions_clean.to_csv('social_media_interactions_cleaned.csv', index=False)

print("Individual cleaned datasets also saved!")


Cleaned and merged dataset saved as: customer_data_cleaned_merged.csv
Individual cleaned datasets also saved!


In [8]:
print("\n" + "="*60)
print("FINAL DATA CLEANING SUMMARY")
print("="*60)

print(f"Original datasets:")
print(f"  Demographics: 3200 rows, 6 columns")
print(f"  Transactions: 3200 rows, 6 columns")
print(f"  Interactions: 3200 rows, 6 columns")

print(f"\nFinal merged dataset: {merged_df.shape[0]} rows, {merged_df.shape[1]} columns")

print(f"\nCleaning actions performed:")
print(f"  ✓ Missing values handled using median/mode imputation")
print(f"  ✓ Duplicates checked and removed (177 demographics, 185 transactions, 181 interactions)")
print(f"  ✓ Date formats standardized")
print(f"  ✓ Categorical data standardized")
print(f"  ✓ Transactions sorted by date (ascending)")
print(f"  ✓ Data integrity validated")
print(f"  ✓ Age outliers corrected (117 unrealistic ages → median 45.0)")
print(f"  ✓ Outliers identified and analyzed")
print(f"  ✓ Datasets successfully merged")

print(f"\nOutlier treatment decisions:")
print(f"  • Age outliers (<15, >95): Corrected to median (45.0)")
print(f"  • Negative/zero amounts: Retained as business refunds/adjustments")
print(f"  • Blank transaction/interaction fields: Retained as valid customer states")

print(f"\nData quality:")
print(f"  • No missing values remaining")
print(f"  • No unrealistic age values remaining")
print(f"  • All date formats consistent")
print(f"  • Data types properly set")
print(f"  • Ready for MS1 analysis!")


FINAL DATA CLEANING SUMMARY
Original datasets:
  Demographics: 3200 rows, 6 columns
  Transactions: 3200 rows, 6 columns
  Interactions: 3200 rows, 6 columns

Final merged dataset: 5728 rows, 16 columns

Cleaning actions performed:
  ✓ Missing values handled using median/mode imputation
  ✓ Duplicates checked and removed (177 demographics, 185 transactions, 181 interactions)
  ✓ Date formats standardized
  ✓ Categorical data standardized
  ✓ Transactions sorted by date (ascending)
  ✓ Data integrity validated
  ✓ Age outliers corrected (117 unrealistic ages → median 45.0)
  ✓ Outliers identified and analyzed
  ✓ Datasets successfully merged

Outlier treatment decisions:
  • Age outliers (<15, >95): Corrected to median (45.0)
  • Negative/zero amounts: Retained as business refunds/adjustments
  • Blank transaction/interaction fields: Retained as valid customer states

Data quality:
  • No missing values remaining
  • No unrealistic age values remaining
  • All date formats consistent
 